In [1]:
import pandas as pd
import numpy as np
from math import trunc
import matplotlib.pyplot as plt
%matplotlib qt
import argparse, sys, os
import pygeodesy
from scipy.signal import butter, filtfilt, resample
from scipy.spatial.distance import cdist
from scipy.interpolate import make_interp_spline, make_smoothing_spline, BSpline
from scipy.ndimage import uniform_filter1d, spline_filter1d
from scipy.ndimage import gaussian_filter1d

In [2]:
# current_path = os.path.dirname(__file__)

#put your relative path to the csv files
raptor_msgs = pd.read_csv('../data/pp_run4_2_raptor.csv')
imu_msgs = pd.read_csv('../data/pp_run4_2_imu.csv')
print(list(raptor_msgs.columns))
print(list(imu_msgs.columns))


['__time', '/raptor_dbw_interface/accelerator_pedal_report/control_type/value', '/raptor_dbw_interface/accelerator_pedal_report/driver_activity', '/raptor_dbw_interface/accelerator_pedal_report/enabled', '/raptor_dbw_interface/accelerator_pedal_report/fault_accel_pedal_system', '/raptor_dbw_interface/accelerator_pedal_report/fault_ch1', '/raptor_dbw_interface/accelerator_pedal_report/fault_ch2', '/raptor_dbw_interface/accelerator_pedal_report/header/stamp/nanosec', '/raptor_dbw_interface/accelerator_pedal_report/header/stamp/sec', '/raptor_dbw_interface/accelerator_pedal_report/ignore_driver', '/raptor_dbw_interface/accelerator_pedal_report/pedal_input', '/raptor_dbw_interface/accelerator_pedal_report/pedal_output', '/raptor_dbw_interface/accelerator_pedal_report/rolling_counter', '/raptor_dbw_interface/accelerator_pedal_report/torque_actual', '/raptor_dbw_interface/brake_2_report/estimated_road_slope', '/raptor_dbw_interface/brake_2_report/front_brake_pressure', '/raptor_dbw_interface

In [3]:
def normalize_data(input_times, data):
    input_times = input_times[~np.isnan(data)]
    data = data[~np.isnan(data)]
    return np.column_stack([input_times, data])

In [4]:
raptor_times = np.array(raptor_msgs['__time'])
imu_times = np.array(imu_msgs['__time'])

raptor_times = np.round(raptor_times - raptor_times[0], 3)
imu_times = np.round(imu_times - imu_times[0], 3)

KPH2MPH = 0.621371
rear_right_wheel_speed = np.array(raptor_msgs['/raptor_dbw_interface/wheel_speed_report/rear_right'])
rear_left_wheel_speed = np.array(raptor_msgs['/raptor_dbw_interface/wheel_speed_report/rear_left'])

rear_avg_wheel_speed = (rear_right_wheel_speed + rear_left_wheel_speed) / 2
desired_vel = np.array(imu_msgs['/vehicle/desired_velocity_readout/data'])
rear_avg_wheel_speed = normalize_data(raptor_times, rear_avg_wheel_speed * KPH2MPH)
desired_vel = normalize_data(imu_times, desired_vel)

filtered_rear_avg_wheel_speed = np.array(rear_avg_wheel_speed)
filtered_rear_avg_wheel_speed[:, 1] = gaussian_filter1d(rear_avg_wheel_speed[:, 1], 10)

KeyError: '/vehicle/desired_velocity_readout/data'

In [5]:
#Wheel speed with residuals
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
axs[0].scatter(rear_avg_wheel_speed[:, 0], rear_avg_wheel_speed[:, 1], s=.5, alpha=0.5, c='orange', label="Rear Avg Velocity")
axs[0].plot(filtered_rear_avg_wheel_speed[:, 0], filtered_rear_avg_wheel_speed[:, 1], 'b', label="Filtered Velocity")
axs[0].plot(desired_vel[:, 0], desired_vel[:, 1], 'r', label="Desired Velocity")
axs[0].legend()
axs[0].set_ylabel("Speed (MPH)")

axs[1].plot(filtered_rear_avg_wheel_speed[4:-2, 0], desired_vel[:, 1] - filtered_rear_avg_wheel_speed[4:-2, 1], 'b', label="Filtered Velocity")
axs[1].legend()
axs[1].set_ylabel("Speed (MPH)")
axs[1].set_xlabel("Time (Seconds)")
axs[1].set_ylim(15, -15)

# print(len(filtered_rear_avg_wheel_speed[4:-2, 0]))
# print(len(desired_vel[:, 0]))
plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [26]:
rear_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/rear_brake_pressure']))
front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
accel_x = normalize_data(imu_times, np.array(imu_msgs['/accel/filtered/accel/accel/linear/x']))
v_accel_x = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/linearaccelbody/x']))

#Deceleration vs Brake Pressure
start = 137000
stop = -25000
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
axs[0].plot(rear_brake_pressure[start:stop, 0], rear_brake_pressure[start:stop, 1] / 1000, 'b', linewidth=0.5, label="Rear Brake Pressure")
axs[0].plot(front_brake_pressure[start:stop, 0], front_brake_pressure[start:stop, 1] / 1000, 'r', linewidth=0.5, label="Front Brake Pressure")
# ax.plot(front_brake_pressure[:, 0], front_brake_pressure[:, 1] / ( rear_brake_pressure[:, 1] + front_brake_pressure[:, 1]) , 'g', label="")
axs[0].legend()
axs[0].set_ylabel("Pressure (Mpa)")
# axs[0].set_xlabel("Time (Seconds)")

axs[1].plot(accel_x[start:stop, 0], accel_x[start:stop, 1], 'b', linewidth=0.5, label="Linear Accel, X")
axs[1].plot(v_accel_x[start:stop, 0], gaussian_filter1d(v_accel_x[start:stop, 1],20), 'r', linewidth=0.5, label="Vectornav Accel, X")
axs[1].legend()
axs[1].set_ylabel("Acceleration (m/s^2)")
axs[1].set_xlabel("Time (Seconds)")
# axs[1].set_ylim(15, -15)
plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/pp_brake_pressure.pdf", format='pdf', dpi=20000)

plt.show()

In [7]:

thr = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/accelerator_pedal_report/pedal_output']))
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))
#Deceleration vs Brake Pressure
start = 60000
stop = -85000
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
MPH2MS = 0.44704
axs[0].scatter(thr[start:stop, 0], thr[start:stop, 1] , s=.5, alpha=.5, c='orange', label="Unfiltered Throttle")
axs[0].plot(filtered_rear_avg_wheel_speed[start:stop, 0], filtered_rear_avg_wheel_speed[start:stop, 1] * MPH2MS, 'orange', label="Velocity")
axs[0].plot(thr[start:stop, 0], gaussian_filter1d(thr[start:stop, 1], 5), 'b', label="Throttle")
axs[0].set_ylim(0, 100)
axs[0].legend()
axs[0].set_ylabel("% and m/s")
# axs[0].set_xlabel("Time (Seconds)")

axs[1].plot(gear[start:stop, 0], gear[start:stop, 1], 'r', label="Gear")
axs[1].legend()
axs[1].set_ylabel("Gear")
axs[1].set_xlabel("Time (Seconds)")
axs[1].set_ylim(0, 6)
plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms_brake_throttle_percent.pdf", format='pdf', dpi=20000)


plt.show()

NameError: name 'filtered_rear_avg_wheel_speed' is not defined